In [1]:
from ngsolve import *
from netgen.geom2d import unit_square
from netgen.csg import *
from mymg import *
from prol import *
from ngsolve.la import EigenValues_Preconditioner
from ngsolve.krylovspace import CGSolver
from netgen.webgui import Draw

In [2]:
# ============ parameters ===========
c_vis, c_div = 1, 1e8
c_lo = 10  # int(sys.argv[3])
dim = 2

# direct = True
# ==================================

if dim == 2:
    mesh = Mesh(unit_square.GenerateMesh(maxh=1 / 4))
    maxdofs = 3e5
else:
    mesh = Mesh(unit_cube.GenerateMesh(maxh=1 / 3))
    maxdofs = 1e5

# ========= NC space
V = FESpace('nonconforming', mesh, dirichlet='.*')


# ======= AL-uzawa solver
fes = V * V
a = BilinearForm(fes)
(ux, uy), (vx, vy) = fes.TnT()
u = CF((ux, uy))
v = CF((vx, vy))
GradU = CF((grad(ux), grad(uy)))
GradV = CF((grad(vx), grad(vy)))
divU = grad(ux)[0] + grad(uy)[1]
divV = grad(vx)[0] + grad(vy)[1]
a += (c_vis * InnerProduct(GradU, GradV) 
      + c_lo * u * v 
      + c_div * divU * divV) * dx

gfu = GridFunction(fes)
uhx, uhy = gfu.components
uh = CF((uhx, uhy))

# ======== MG components
et = meshTopology(mesh, mesh.dim)
et.Update()
prolV = FacetProlongationTrig2(mesh, et)
a.Assemble()
pre = MultiGrid(a.mat, prolV, nc=V.ndof,
                    coarsedofs=fes.FreeDofs(), w1=0.8,
                    nsmooth=4, sm="gs", var=True,
                    he=True, dim=dim, wcycle=False)

In [3]:
# ===== exact solution
u_exact1 = x ** 2 * (x - 1) ** 2 * 2 * y * (1 - y) * (2 * y - 1)
u_exact2 = y ** 2 * (y - 1) ** 2 * 2 * x * (x - 1) * (2 * x - 1)
u_exact = CF((u_exact1, u_exact2))

L_exactXX = c_vis * (2 * x * (x - 1) ** 2 * 2 * y * (1 - y) * (2 * y - 1)
                     + x ** 2 * 2 * (x - 1) * 2 * y * (1 - y) * (2 * y - 1))
L_exactXY = c_vis * (x ** 2 * (x - 1) ** 2 * 2 * (1 - y) * (2 * y - 1)
                     - x ** 2 * (x - 1) ** 2 * 2 * y * (2 * y - 1)
                     + 2 * x ** 2 * (x - 1) ** 2 * 2 * y * (1 - y))
L_exactYX = c_vis * (y ** 2 * (y - 1) ** 2 * 2 * (x - 1) * (2 * x - 1)
                     + y ** 2 * (y - 1) ** 2 * 2 * x * (2 * x - 1)
                     + 2 * y ** 2 * (y - 1) ** 2 * 2 * x * (x - 1))
L_exactYY = c_vis * (2 * y * (y - 1) ** 2 * 2 * x * (x - 1) * (2 * x - 1)
                     + y ** 2 * 2 * (y - 1) * 2 * x * (x - 1) * (2 * x - 1))
L_exact = CF((L_exactXX, L_exactXY, L_exactYX, L_exactYY), dims=(2, 2))

p_exact = -100 * x * (1 - x) * (1 - y) - 100 / 12


# ===== linear forms
f = LinearForm(fes)
f += (-c_vis * (4 * y * (1 - y) * (2 * y - 1) * ((1 - 2 * x) ** 2 - 2 * x * (1 - x))
                + 12 * x ** 2 * (1 - x) ** 2 * (1 - 2 * y))
      + 100 *(1 - 2 * x) * (1 - y)) * v[0] * dx
f += (-c_vis * (4 * x * (1 - x) * (1 - 2 * x) * ((1 - 2 * y) ** 2 - 2 * y * (1 - y))
                + 12 * y ** 2 * (1 - y) ** 2 * (2 * x - 1))
      - 100 * x * (1 - x)) * v[1] * dx
f += c_lo * u_exact * v * dx

In [4]:
def SolveBVP(level):
    fes.Update()
    gfu.Update()
    a.Assemble()
    f.Assemble()
    
    # # update MG
    if level > 0:
        et.Update()
        pp = [fes.FreeDofs()]
        pp.append(V.ndof)
        pdofs = BitArray(fes.ndof)
        pdofs[:] = 0
        inner = prolV.GetInnerDofs(level)
        for j in range(dim):
            pdofs[j * V.ndof:(j + 1) * V.ndof] = inner
        # he_prol
        pp.append(a.mat.Inverse(pdofs, inverse="umfpack"))
        # bk smoother
        # TODO: WHY THIS IS WRONG???!!!
        # bjac = et.CreateSmoother(a, {"blocktype": "vertexpatch"})
        # pp.append(bjac)
        pp.append(VertexPatchBlocks(mesh, fes))
        # pp.append(fes.FreeDofs())
        pre.Update(a.mat, pp)

    # dirichlet BC
    # homogeneous Dirichlet assumed
    # ===== one-time AL-uzawa solver, epsilon = 1/c_div
    lams = EigenValues_Preconditioner(mat=a.mat, pre=pre)
    inv = CGSolver(a.mat, pre, printrates=False, tol=1e-8, maxiter=1000)
    # inv = a.mat.Inverse(fes.FreeDofs(), inverse='umfpack')
    # it = 1
    gfu.vec.data = inv * f.vec
    it = inv.iterations
    # print(f'err P norm: {Norm(errP):.2E}, Err_P norm decrease rate: {Norm(errP) / Norm(errP0):.1E}')
    print(f"MAX PREC LAM: {max(lams):.1E}; MIN PREC LAM: {min(lams):.1E}")
    print("IT: %2.0i" % (it), "cond: %.2e" % (max(lams) / min(lams)), "NDOFS: %.2e" % (
        sum(fes.FreeDofs())))

In [5]:
# SolveBVP(0)
u_rate = 0
level = 1
while True:
    with TaskManager():
        mesh.ngmesh.Refine()
        # exit if total global dofs exceed a tol
        V.Update()
        if (V.ndof * dim > maxdofs):
            print(V.ndof * dim)
            break
        if level == 1:
            print('=====================')
            print(
                f"DIM: {dim}, c_vis: {c_vis:.1E}, c_low: {c_lo:.1E}, 1/eps: {c_div:.1E}")
            print('=====================')
        print(f'level: {level}')
        SolveBVP(level)
        # ===== convergence check =====
        meshRate = 2
        L2_uErr = sqrt(Integrate((uh - u_exact) * (uh - u_exact), mesh))
        divUh = grad(uhx)[0] + grad(uhy)[1]
        L2_divErr = sqrt(Integrate(divUh * divUh, mesh))
        if level != 1:
            u_rate = log(prev_uErr / L2_uErr) / log(meshRate)
        print(f"uh L2-error: {L2_uErr:.3E}, uh conv rate: {u_rate:.2E}")
        print(f"uh divErr: {L2_divErr:.1E}, 1/epsilon: {c_div:.1E}")
        print('==============================')
        prev_uErr = L2_uErr
        level += 1

DIM: 2, c_vis: 1.0E+00, c_low: 1.0E+01, 1/eps: 1.0E+08
level: 1
MAX PREC LAM: 1.3E+00; MIN PREC LAM: 1.0E+00
IT:  7 cond: 1.30e+00 NDOFS: 3.76e+02
uh L2-error: 4.878E-02, uh conv rate: 0.00E+00
uh divErr: 6.2E-08, 1/epsilon: 1.0E+08
level: 2
MAX PREC LAM: 1.5E+00; MIN PREC LAM: 1.0E+00
IT:  9 cond: 1.45e+00 NDOFS: 1.57e+03
uh L2-error: 1.399E-02, uh conv rate: 1.80E+00
uh divErr: 6.4E-08, 1/epsilon: 1.0E+08
level: 3
MAX PREC LAM: 1.5E+00; MIN PREC LAM: 1.0E+00
IT:  9 cond: 1.54e+00 NDOFS: 6.40e+03
uh L2-error: 3.709E-03, uh conv rate: 1.92E+00
uh divErr: 6.4E-08, 1/epsilon: 1.0E+08
level: 4
MAX PREC LAM: 1.7E+00; MIN PREC LAM: 9.9E-01
IT: 10 cond: 1.71e+00 NDOFS: 2.59e+04
uh L2-error: 9.467E-04, uh conv rate: 1.97E+00
uh divErr: 6.4E-08, 1/epsilon: 1.0E+08
level: 5
MAX PREC LAM: 1.8E+00; MIN PREC LAM: 9.3E-01
IT: 10 cond: 1.94e+00 NDOFS: 1.04e+05
uh L2-error: 2.383E-04, uh conv rate: 1.99E+00
uh divErr: 6.5E-08, 1/epsilon: 1.0E+08
559054
